In [21]:
# Import libraries
import datetime
import functools
import matplotlib.pyplot as plt
import numpy as np
import operator
import pandas as pd
pd.set_option("display.max_rows", 500)
pd.options.display.max_colwidth = 1000
import seaborn as sns

In [22]:
# .csv path
hotels_path = "../data/clean_hotels_scraped_v2.csv"
trivago_path = "../data/train.csv"

# Create dataframe
trivago_df = pd.read_csv(trivago_path)
hotels_df = pd.read_csv(hotels_path, usecols = ["city", "country", "hotel_name", "rating", 
                                                "address", "popularity_rating", "locality", "price",
                                               "landmark", "URL"])

# Sanity check
hotels_df.head()

,city,country,hotel_name,rating,address,popularity_rating,locality,price,landmark,URL
0,Mutterstadt,Germany,RheinCity Hotel,4.0,"Zollhofstraße 11, Ludwigshafen, RP, 67059, Germany",4.0,Ludwigshafen,107.0,"['5.4 miles to mutterstadt, 3.3 miles to mannheim (mhg)']",https://www.hotels.com/ho804141952/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=1&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
1,Mutterstadt,Germany,NYX Hotel Mannheim by Leonardo Hotels,4.0,"F4, 4-11, Mannheim, BW, 68159, Germany",14.0,Mannheim,78.0,"['6.2 miles to mutterstadt, 2.8 miles to mannheim (mhg)']",https://www.hotels.com/ho408334/?q-check-out=2020-08-29&FPQ=2&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=2&tab=description&JHR=2&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
2,Mutterstadt,Germany,"Radisson Blu Hotel, Mannheim",4.5,"Quadrant Q7, 27, Mannheim, 68161, Germany",387.0,Mannheim,130.0,"['6.4 miles to mutterstadt, 2.3 miles to mannheim (mhg)']",https://www.hotels.com/ho626280/?pa=3&tab=description&q-room-0-adults=2&intlid=SoldOutListing&ZSX=0&SYE=3&q-room-0-children=0
3,Mutterstadt,Germany,ACHAT Comfort Frankenthal/Pfalz,3.5,"Mahlastrasse 18, Frankenthal, RP, 67227, Germany",64.0,Frankenthal,83.0,"['5.9 miles to mutterstadt, 8.3 miles to mannheim (mhg)']",https://www.hotels.com/ho180949/?q-check-out=2020-08-29&FPQ=2&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=4&tab=description&JHR=2&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
4,Mutterstadt,Germany,Relax Apartment,4.0,"F4 14-15, Mannheim, 68159, Germany",175.0,Mannheim,130.0,"['6.2 miles to mutterstadt, 2.7 miles to mannheim (mhg)']",https://www.hotels.com/ho685840/?q-check-out=2020-08-29&FPQ=2&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=5&tab=description&JHR=2&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3


In [23]:
# Sanity check
trivago_df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [24]:
# Most popular destination
most_popular_cities = trivago_df.city.value_counts().iloc[:100].index

In [25]:
most_popular_cities = list(most_popular_cities)

In [26]:
cities = []
for city in most_popular_cities:
    string = city.split(",")
    cities.append(string[0])
print(cities)

['London', 'Paris', 'Istanbul', 'New York', 'Rio de Janeiro', 'Amsterdam', 'Rome', 'Cancun', 'Tokyo', 'Berlin', 'Dubai', 'Madrid', 'Las Vegas', 'Barcelona', 'Prague', 'São Paulo', 'Natal', 'Florianópolis', 'Buenos Aires', 'Orlando', 'Bangkok', 'Kuala Lumpur', 'Miami Beach', 'Vienna', 'Porto Seguro', 'Milan', 'Foz do Iguaçu', 'Playa del Carmen', 'Osaka', 'Hamburg', 'Sydney', 'Armação dos Búzios', 'Melbourne', 'Maceió', 'Fortaleza', 'Porto de Galinhas', 'Acapulco', 'Lisbon', 'Singapore', 'Salvador Bahia', 'Puerto Vallarta', 'Gramado', 'Hong Kong', 'Malacca', 'Mexico City', 'Guarujá', 'Ubatuba', 'Munich', 'Venice', 'Budapest', 'Cartagena', 'Moscow', 'Edinburgh', 'Marrakech', 'Kyoto', 'Port Dickson', 'Campos do Jordão', 'Florence', 'Balneário Camboriú', 'Paraty', 'Antalya', 'Mazatlán', 'Caldas Novas', 'San Francisco', 'Dublin', 'Copenhagen', 'Cabo Frio', 'Mumbai', 'Cologne', 'Ankara', 'Guadalajara', 'Ilhabela', 'Seoul', 'Arraial do Cabo', 'Udaipur', 'Seville', 'Taipei City', 'St Petersburg

In [27]:
# Best reviews
best_reviews = hotels_df[hotels_df["rating"] >= 4.5]
best_reviews = best_reviews[best_reviews["popularity_rating"] > 50]

In [28]:
best_reviews

,city,country,hotel_name,rating,address,popularity_rating,locality,price,landmark,URL
2,Mutterstadt,Germany,"Radisson Blu Hotel, Mannheim",4.5,"Quadrant Q7, 27, Mannheim, 68161, Germany",387.0,Mannheim,130.0,"['6.4 miles to mutterstadt, 2.3 miles to mannheim (mhg)']",https://www.hotels.com/ho626280/?pa=3&tab=description&q-room-0-adults=2&intlid=SoldOutListing&ZSX=0&SYE=3&q-room-0-children=0
12,Mutterstadt,Germany,BoardingHouse Mannheim,4.5,"26 S6, Mannheim, 68161, Germany",53.0,Mannheim,108.0,"['6.5 miles to mutterstadt, 2.2 miles to mannheim (mhg)']",https://www.hotels.com/ho618455328/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=13&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
31,Mutterstadt,Germany,Qube Heidelberg,4.5,"Bergheimer Str. 74, Heidelberg, BW, 69115, Germany",595.0,Heidelberg,153.0,"['15 miles to mutterstadt, 8.6 miles to mannheim (mhg)']",https://www.hotels.com/ho335220/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=32&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
33,Mutterstadt,Germany,Hotel Europäischer Hof Heidelberg,4.5,"Friedrich-Ebert-Anlage 1, Heidelberg, BW, 69011, Germany",584.0,Heidelberg,251.0,"['16 miles to mutterstadt, 9.1 miles to mannheim (mhg)']",https://www.hotels.com/ho376787/?q-check-out=2020-08-29&FPQ=4&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=34&tab=description&JHR=5&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
55,Mutterstadt,Germany,Bacchus Hotel,4.5,"Rodensteinstrasse 30, Bensheim, 64625, Germany",158.0,Bensheim,134.0,"['21 miles to mutterstadt, 15 miles to mannheim (mhg)']",https://www.hotels.com/ho883080640/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=56&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
78,Passau,Germany,Landgasthof Euler,4.5,"Kaiserstraße 10, Neuschoenau, BY, 94556, Germany",52.0,Neuschoenau,88.0,"['21 miles to passau, 21 miles to passau central station']",https://www.hotels.com/ho1008295072/?q-check-out=2020-08-29&FPQ=2&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=17&tab=description&JHR=2&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
91,Hamburg,Germany,Hyperion Hotel Hamburg,4.5,"Amsinckstraße 39, Hamburg, 20097, Germany",677.0,Hammerbrook,116.0,"['0.9 miles to city center, 5.9 miles to hamburg airport (ham)']",https://www.hotels.com/ho534642/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=6&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
94,Hamburg,Germany,Clipper Elb-Lodge,4.5,"Carsten-Rehder-Strasse 71, Hamburg, HH, 22767, Germany",500.0,Hamburg-Altona,155.0,"['1.9 miles to city center, 6.6 miles to hamburg airport (ham)']",https://www.hotels.com/ho428977/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=9&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
95,Hamburg,Germany,Hotel Wedina an der Alster,4.5,"Gurlittstraße 23, Hamburg, HH, 20099, Germany",297.0,Saint Georg,143.0,"['0.8 miles to city center, 5.2 miles to hamburg airport (ham)']",https://www.hotels.com/ho509812/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=10&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
97,Hamburg,Germany,Hanse Clipper Haus,4.5,"Ditmar-Koel-Strasse 1, Hamburg, HH, 20459, Germany",712.0,Portuguese Quarter,153.0,"['0.7 miles to city center, 6.1 miles to hamburg airport (ham)']",https://www.hotels.com/ho428578/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=12&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3


In [29]:
best_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72118 entries, 2 to 422375
Data columns (total 10 columns):
city                 72118 non-null object
country              72118 non-null object
hotel_name           72118 non-null object
rating               72118 non-null float64
address              72118 non-null object
popularity_rating    72118 non-null float64
locality             72118 non-null object
price                72118 non-null float64
landmark             72118 non-null object
URL                  72118 non-null object
dtypes: float64(3), object(7)
memory usage: 6.1+ MB


In [30]:
# Sort best reviews by popularity rating and price
best_reviews = best_reviews.sort_values(by = ["popularity_rating"], axis = 0, 
                         ascending = False)

In [31]:
# Reset index
best_reviews = best_reviews.reset_index(drop = True)
best_reviews.head()

,city,country,hotel_name,rating,address,popularity_rating,locality,price,landmark,URL
0,Choisy-le-Roi,France,Hôtel Henriette,4.5,"9 Rue des Gobelins, Paris, Paris, 75013, France",999.0,Paris,130.0,"['5.8 miles to choisy-le-roi, 15 miles to roissy - charles de gaulle airport (cdg)']",https://www.hotels.com/ho448748/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=41&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
1,Treasure Island,USA,Barefoot Beach Club,4.5,"13238 Gulf Blvd, Madeira Beach, FL, 33708, United States",999.0,Madeira Beach,279.0,"['1.9 miles to treasure island, 20 miles to tampa, fl (tpa-tampa intl.)']",https://www.hotels.com/ho247119/?q-check-out=2020-08-29&FPQ=4&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=22&tab=description&JHR=5&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
2,Ellenton,USA,Barefoot Beach Club,4.5,"13238 Gulf Blvd, Madeira Beach, FL, 33708, United States",999.0,Madeira Beach,279.0,"['25 miles to ellenton, 31 miles to sarasota, fl (srq-sarasota-bradenton intl.)']",https://www.hotels.com/ho247119/?q-check-out=2020-08-29&FPQ=4&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=46&tab=description&JHR=5&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
3,Redington Shores,USA,Barefoot Beach Club,4.5,"13238 Gulf Blvd, Madeira Beach, FL, 33708, United States",999.0,Madeira Beach,279.0,"['4.0 miles to redington shores, 20 miles to tampa, fl (tpa-tampa intl.)']",https://www.hotels.com/ho247119/?q-check-out=2020-08-29&FPQ=4&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=33&tab=description&JHR=5&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3
4,Gentilly,France,Hôtel Henriette,4.5,"9 Rue des Gobelins, Paris, Paris, 75013, France",999.0,Paris,130.0,"['1.6 miles to gentilly, 15 miles to roissy - charles de gaulle airport (cdg)']",https://www.hotels.com/ho448748/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=7&tab=description&JHR=4&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3


In [32]:
# Unpersonalized recommendations
def recommendation_baseline():
    # Get list of most popular cities
    cities = ['London', 'Paris', 'Istanbul', 'New York', 'Rio de Janeiro', 'Amsterdam', 'Rome', 'Cancun', 
              'Tokyo', 'Berlin', 'Dubai', 'Madrid', 'Las Vegas', 'Barcelona', 'Prague', 'São Paulo', 'Natal', 
              'Florianópolis', 'Buenos Aires', 'Orlando', 'Bangkok', 'Kuala Lumpur', 'Miami Beach', 'Vienna', 
              'Porto Seguro', 'Milan', 'Foz do Iguaçu', 'Playa del Carmen', 'Osaka', 'Hamburg', 'Sydney', 
              'Armação dos Búzios','Melbourne', 'Maceió', 'Fortaleza', 'Porto de Galinhas', 'Acapulco', 'Lisbon', 
              'Singapore', 'Salvador Bahia','Puerto Vallarta', 'Gramado', 'Hong Kong', 
              'Malacca', 'Mexico City', 'Guarujá', 'Ubatuba', 'Munich', 'Venice', 
              'Budapest', 'Cartagena', 'Moscow', 'Edinburgh', 'Marrakech', 'Kyoto', 'Port Dickson', 
              'Campos do Jordão','Florence', 'Balneário Camboriú', 'Paraty', 'Antalya', 'Mazatlán', 
              'Caldas Novas', 'San Francisco', 'Dublin','Copenhagen', 'Cabo Frio', 'Mumbai', 
              'Cologne', 'Ankara', 'Guadalajara', 'Ilhabela', 'Seoul', 'Arraial do Cabo', 
              'Udaipur', 'Seville', 'Taipei City', 'St Petersburg', 'Miami', 'Québec City', 
              'Morro de São Paulo', 'Curitiba','Athens', 'João Pessoa', 'São Sebastião', 
              'Pattaya', 'Los Angeles', 'Bombinhas', 'Urayasu', 'Manchester','Praia da Pipa', 
              'Brussels', 'Brisbane', 'Lake Buena Vista', 'Stockholm', 'Frankfurt', 'Auckland', 
              'Anaheim', 'Fukuoka', 'Fort Lauderdale']
    
    indexes = []
    
    # Best reviews
    best_reviews = hotels_df[hotels_df["rating"] >= 4.5]
    best_reviews = best_reviews[best_reviews["popularity_rating"] > 100]
    
    # Sort best reviews by popularity rating and price
    best_reviews = best_reviews.sort_values(by = ["popularity_rating"], axis = 0, 
                         ascending = False)
    # Reset index
    best_reviews = best_reviews.reset_index(drop = True)
    
    for city in cities:
        # Get the hotels with 5.0 ratings that are at each of the top cities
        idxs = best_reviews[best_reviews["city"] == city].index
        indexes.append(idxs)
        
    flatten = functools.reduce(operator.iconcat, indexes, [])
    # Create lists to then create json object
    name = []
    city = []
    country = []
    url = []
    landmark = []
    locality = []
    rating = []
    popularity = []
    # Loop through all idxs in the flattened id list, and add the info at each idx row to list
    for i, idx in enumerate(flatten[:31]):
        city.append(best_reviews.at[idx, "city"])
        country.append(best_reviews.at[idx, "country"])
        name.append(best_reviews.at[idx, "hotel_name"])
        rating.append(best_reviews.at[idx, "rating"])
        popularity.append(best_reviews.at[idx, "popularity_rating"])
        landmark.append(best_reviews.at[idx, "landmark"])
        locality.append(best_reviews.at[idx, "locality"])
        url.append(best_reviews.at[idx, "URL"])
    # Create json object
    hotels = [{"name": n, "city": c + ", " + p, "url": u, "landmark": l,
                    "locality": t, "rating": r} for n, c, p, u, l, t, r 
                  in zip(name, city, country, url, landmark, locality, rating)]
    return hotels

    
    

In [33]:
recommendations = recommendation_baseline()

In [34]:
recommendations

[{'city': 'London, Canada',
  'landmark': "['4.0 miles to university of western ontario, 1.8 miles to western fair entertainment centre']",
  'locality': 'South London',
  'name': 'Country Inn & Suites by Radisson, London South, ON',
  'rating': 4.5,
  'url': 'https://www.hotels.com/ho248240/?q-check-out=2020-08-29&FPQ=2&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=2&tab=description&JHR=2&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3'},
 {'city': 'London, Canada',
  'landmark': "['1.6 miles to university of western ontario, 1.6 miles to western fair entertainment centre']",
  'locality': 'Downtown London',
  'name': 'The Park Hotel London',
  'rating': 4.5,
  'url': 'https://www.hotels.com/ho123998/?pa=3&tab=description&q-room-0-adults=2&intlid=SoldOutListing&ZSX=0&SYE=3&q-room-0-children=0'},
 {'city': 'Paris, France',
  'landmark': "['1.2 miles to city center, 3.0 miles to eiffel tower']",
  'locality': 'Quartier de Croulebarbe',
  'name': 'Hôtel Henriette',
  'rating': 4.5,

In [18]:
# Best reviews_2
best_reviews_2 = hotels_df[hotels_df["rating"] >= 4.0]
best_reviews_2 = best_reviews_2.sort_values(by = ["popularity_rating", "rating", "price"], axis = 0, 
                         ascending = [False, False, True])
best_reviews_2 = best_reviews_2.reset_index(drop = True)

In [48]:
# Unpersonalized recommendations
def citybased_recommendation_baseline(location):
    """ Returns top hotel recommendations for a """
    
    location = location.split(",")
    city = location[0].strip()
    country = location[1]
    # Get the hotels that satisfy the keyword
    subset_of_df = best_reviews_2[best_reviews_2["city"] == city]
    idxs = list(subset_of_df[subset_of_df["country"] == country].index)
    # Create lists to store results
    name = []
    city = []
    country = []
    url = []
    landmark = []
    locality = []
    rating = []
    popularity = []
    # Loop through all idxs in the flattened id list, and add the info at each idx row to dictionary
    for idx in idxs[:26]:
        city.append(best_reviews_2.at[idx, "city"])
        country.append(best_reviews_2.at[idx, "country"])
        name.append(best_reviews_2.at[idx, "hotel_name"])
        rating.append(best_reviews_2.at[idx, "rating"])
        popularity.append(best_reviews_2.at[idx, "popularity_rating"])
        landmark.append(best_reviews_2.at[idx, "landmark"])
        locality.append(best_reviews_2.at[idx, "locality"])
        url.append(best_reviews_2.at[idx, "URL"])
    hotels = [{"name": n, "city": c + ", " + p, "url": u, "landmark": l,
                    "locality": t, "rating": r} for n, c, p, u, l, t, r 
                  in zip(name, city, country, url, landmark, locality, rating)]
    return hotels

In [49]:
citybased_recommendation_baseline("London, United Kingdom")

[{'city': 'London,  United Kingdom',
  'landmark': "['0.6 miles to city center, 15 miles to heathrow airport (lhr)']",
  'locality': 'South Bank',
  'name': 'Park Plaza Westminster Bridge London',
  'rating': 4.0,
  'url': 'https://www.hotels.com/ho335698/?pa=3&tab=description&q-room-0-adults=2&intlid=SoldOutListing&ZSX=0&SYE=3&q-room-0-children=0'},
 {'city': 'London,  United Kingdom',
  'landmark': "['1.1 miles to city center, 15 miles to heathrow airport (lhr)']",
  'locality': 'Newington',
  'name': 'H10 London Waterloo',
  'rating': 4.0,
  'url': 'https://www.hotels.com/ho337606/?q-check-out=2020-08-29&FPQ=3&q-check-in=2020-08-27&WOE=6&WOD=4&q-room-0-children=0&pa=2&tab=description&JHR=3&q-room-0-adults=2&YGF=14&MGT=2&ZSX=0&SYE=3'},
 {'city': 'London,  United Kingdom',
  'landmark': "['2.5 miles to city center, 12 miles to heathrow airport (lhr)']",
  'locality': 'South Kensington',
  'name': 'Radisson Blu Edwardian Vanderbilt Hotel',
  'rating': 4.0,
  'url': 'https://www.hotels.